# 목표

- 관세청 영문 문장 데이터 추가

# Library

In [1]:
# visualization
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'.venv/Lib/site-packages/matplotlib/mpl-data/fonts/ttf/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import missingno as msno

In [2]:
import os
import re
import time
import pandas as pd

from tqdm import tqdm

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By

# Data Load

In [3]:
text_data = pd.read_csv('./data/영문_텍스트.csv', dtype=str)
statistics_data = pd.read_csv('./data/통계청.csv', dtype=str)
customs_data = pd.read_csv('./data/관세청.csv', dtype=str)

In [4]:
text_data_copy = text_data.copy()
statistics_data_copy = statistics_data.copy()
customs_data_copy = customs_data.copy()

In [5]:
display(text_data_copy.info())
display(statistics_data_copy.info())
display(customs_data_copy.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      10000 non-null  object
 1   CODE    10000 non-null  object
 2   DSC     10000 non-null  object
dtypes: object(3)
memory usage: 234.5+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6848 entries, 0 to 6847
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ISIC4_국제표준산업분류     2394 non-null   object
 1   ISIC4_분류명          2393 non-null   object
 2   KSIC10_한국표준산업분류    2933 non-null   object
 3   KSIC10_분류명         2930 non-null   object
 4   HS2017_관세통계통합품목분류  5637 non-null   object
 5   HS2017_분류명         5637 non-null   object
dtypes: object(6)
memory usage: 321.1+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12422 entries, 0 to 12421
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   HS부호       12422 non-null  object
 1   한글품목명      12422 non-null  object
 2   영문품목명      12422 non-null  object
 3   성질통합분류코드   11294 non-null  object
 4   성질통합분류코드명  11294 non-null  object
dtypes: object(5)
memory usage: 485.4+ KB


None

In [21]:
customs_data_copy

,HS부호,한글품목명,영문품목명,성질통합분류코드,성질통합분류코드명
0,0101211000,농가 사육용,For farm breeding,11020101,말
1,0101219000,기타,Other,11020101,말
2,0101291000,경주말,Horses for racing,11020101,말
3,0101299000,기타,Other,11020101,말
4,0101300000,당나귀,Asses,11020190,기타 산 동물
...,...,...,...,...,...
12417,9706102000,악기류,Musical instruments,12050201,수집품 및 골동품
12418,9706103000,기타,Other,12050201,수집품 및 골동품
12419,9706901000,도자기류,Ceramics,12050201,수집품 및 골동품
12420,9706902000,악기류,Musical instruments,12050201,수집품 및 골동품


In [6]:
# -------------------- 함수 모음 --------------------
## 스크롤 끝까지
def scroll(driver):
    scroll_location = driver.execute_script('return document.body.scrollHeight')
    while True:
        driver.execute_script('window.scrollTo(0,document.body.scrollHeight)')
        time.sleep(2)
        scroll_height = driver.execute_script('return document.body.scrollHeight')
        if scroll_location == scroll_height:
            break
        else:
            scroll_location = driver.execute_script('return document.body.scrollHeight')
    driver.implicitly_wait(3)


## 스크롤 한 번만
def scroll_one(driver):
    driver.execute_script('window.scrollTo(0,document.body.scrollHeight)')
    time.sleep(1)
    driver.implicitly_wait(3)

In [7]:
# -------------------- 크롬 준비 --------------------
options = Options()
options.add_experimental_option("detach", True) # 브라우저 꺼짐 방지

service = Service(excutable_path=ChromeDriverManager().install())

In [9]:
driver = webdriver.Chrome(service=service, options=options)
url = 'https://unipass.customs.go.kr/clip/index.do'
driver.get(url=url)

In [10]:
# 세계HS 탭 클릭

# #topmenu > ul > li:nth-child(3)
# //*[@id="topmenu"]/ul/li[3]

btn = driver.find_element(By.XPATH, '//*[@id="topmenu"]/ul/li[3]')
btn.click()

In [13]:
# 관세율표 클릭

btn = driver.find_element(By.XPATH, '//*[@id="leftmenu"]/ul/li[1]/ul/li[3]')
btn.click()

In [20]:
# 2017년 클릭

btn = driver.find_element(By.XPATH, '//*[@id="searchVo"]/ul/li[1]/ul/li[8]/a')
btn.click()

In [22]:
# 검색어 입력

inp = driver.find_element(By.XPATH, '//*[@id="searchVal"]')
inp.send_keys('0101211000')

In [23]:
# 검색 버튼 클릭

btn = driver.find_element(By.XPATH, '//*[@id="searchVo"]/div/span/button')
btn.click()

In [24]:
scroll_one(driver)

In [27]:
# 국문 텍스트 전체 수집

parse = driver.find_element(By.XPATH, '//*[@id="divLft_tab4"]').text
print(parse)

01.01 - 살아 있는 말ㆍ당나귀ㆍ노새ㆍ버새(+)
- 말
0101.21 - 번식용
0101.29 - 기타
0101.30 - 당나귀
0101.90 - 기타
이 호에는 말(암컷의 말․번식용 말․거세한 말․새끼말․조랑말)․당나귀․노새와 버새를 분류한다(야생이나 가축인지에 상관없다).
노새는 당나귀와 암컷 말과의 잡종이며, 버새는 번식용 말과 당나귀와의 교배종이다.
◦
◦ ◦
[소호해설]
소호 제0101.21호
소호 제0101.21호에서 “번식용 동물(pure-bred breeding animals)”에는 소관 국가기관에서 “순종(pure-bred)”으로 간주하는 번식용 동물로 한정하여 분류한다.


In [28]:
# 영문 버튼 클릭

btn = driver.find_element(By.XPATH, '//*[@id="tab4"]/div/div[1]/span/button[2]')
btn.click()

In [29]:
# 영문 텍스트 전체 수집

parse = driver.find_element(By.XPATH, '//*[@id="divLft_tab4"]').text
print(parse)

01.01 Live horses, asses, mules and hinnies (+).
- Horses :
0101.21 - Pure-bred breeding animals
0101.29 - Other
0101.30 - Asses
0101.90 - Other
This heading covers horses (mares, stallions, geldings, foals and ponies), asses, mules and hinnies, whether domestic or wild.
Mules are the hybrid offspring of the ass and the mare. The hinny is bred from the stallion and the ass.
°
° °
Subheading Explanatory Note.
Subheading 0101.21
For the purposes of subheading 0101.21, the expression “ pure bred breeding animals ” covers only those breeding animals which are regarded as “ pure bred ” by the competent national authorities.
